In [290]:
import torch
import torch.nn as nn
import numpy as np
import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

#### Hyper-params

In [234]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [235]:
device

device(type='cuda', index=0)

#### Text Preprocessing

In [236]:
# read the text file
text = open("austen-emma.txt", "r").read()

In [238]:
## tokenization
chars = tuple(set(text))
idx2char = dict(enumerate(chars))
char2idx = dict(zip(idx2char.values(), idx2char.keys()))
#  encode the text into tokens
encoded = np.array([char2idx[ch] for ch in text])

In [300]:
## Convert the tokens into one-hot encoding
def one_hot_encoding(data, n_labels):
    
#     print(encoding.shape)
    # zeros matrix of size mx/n  where m --> size of encoding and n--> size of 
    one_hot = np.zeros((data.shape[0], n_labels), dtype=np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), data.flatten()] = 1.

    return one_hot
#     # file the token with 1s
#     for i, token in enumerate(encoding):
#         one_hot[i][token] = 1
#     return one_hot
    

In [301]:
one_hot_encoding(np.array([1, 2, 3]), 9).shape

(3, 9)

In [321]:
def load_data(encoded, seq_length = 100):
    
    # keep only valid charas
    sequences = len(encoded)//(seq_len+1)
    encoded = encoded[:sequences*(seq_len+1)]
    
    x = np.empty((sequences, seq_len, len(chars)), dtype=np.float32)
    y = np.empty((sequences, seq_len), dtype=np.float32)
    
    print(x.shape, y.shape)
    
    for i, current in enumerate(range(0, encoded.shape[0], seq_len + 1)):
        x[i] = one_hot_encoding(np.array(encoded[current:current+seq_len]), n_labels=len(chars))
        y[i] = np.array(encoded[current+1:current+seq_len+1])
    return x, y
    

In [322]:
x, y = load_data(encoded)

(13647, 64, 77) (13647, 64)


In [323]:
print(x.shape, y.shape)
print(x.dtype, y.dtype)

(13647, 64, 77) (13647, 64)
float32 float32


In [324]:
class CustomDataset(Dataset):
    
    def __init__(self):
        self.x, self.y = load_data(encoded, seq_len)
        self.len = x.shape[0]
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    

In [325]:
# data 
seq_len = 64
batch_size = 64

In [326]:
dataset = CustomDataset()
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

(13647, 64, 77) (13647, 64)


In [327]:
# see the text of each seq 
x, y = next(iter(dataloader))
s = ""
for each in x[0]:
    s = s +""+ idx2char[torch.argmax(each).item()]
s

'od opinion.--He had been sitting with them\nhalf an hour, she fou'

### RNN Model

In [388]:
class RNN(nn.Module):
    
    def __init__(self, input_size, output_size, hidden_dim=64, n_layers=1,dropout=0.20):
        
        super(RNN, self).__init__()
        self.n_hidden = hidden_dim
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.dropout = nn.Dropout(p=dropout)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=output_size)
        
    def forward(self, x, hidden):
        
        lstm_out, hidden = self.lstm(x, hidden)
        out = self.dropout(lstm_out)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden    

#### Params configuration model and training

In [401]:
# model
input_size = len(chars)
hidden_dim = 64
n_layers = 1
output_size = len(chars)

epochs = 50
lr = 1e-2

writer = SummaryWriter(log_dir="./runs")
# training

In [402]:
# create the model
model = RNN(input_size=input_size, output_size=output_size).to(device)

In [403]:
### Optimizer and Loss
optmizer = torch.optim.Adam(params=model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

In [404]:
### Configurations

epoch_progress = tqdm.tqdm(total=epochs, desc="Epoch", position=0)

steps = 0

for epoch in range(epochs):
    
    batch_progress = tqdm.tqdm(total=len(dataloader), desc="Batch", position=0)
    epoch_loss = []

    
    for i, batch in enumerate(dataloader):
        x, y = batch
        
        x = x.to(device)
        y = y.to(device)
        
        
        outputs, _ = model(x, None)
        
#         print(x.size(), y.size(), outputs.size(), y.view(batch_size, seq_len).size(0))
        
        loss = criterion(outputs, y.view(-1).long())
        
        # backpropagate the loss update the params
        optmizer.zero_grad()
        loss.backward()
        optmizer.step()
        
        if steps%100==0:
            print(f"Epoch {epoch} | Steps {steps} | Loss = {loss.item()}")
        
        writer.add_scalar("Step Wise Loss", loss.item(), steps)
        epoch_loss.append(loss.item())
        
        batch_progress.update(1)
        steps += 1
    
    avg_loss = sum(epoch_loss)/len(epoch_loss)
    writer.add_scalar("Epoch Loss", avg_loss, epoch)
    if epoch%(epochs//10)==0:
        print(f"Epoch = {epoch} | Avg Loss = {avg_loss}")
    epoch_progress.update(1)


Batch:  12%|█▏        | 26/214 [00:00<00:01, 119.85it/s]

Epoch 0 | Steps 0 | Loss = 4.34761905670166


Batch:  55%|█████▌    | 118/214 [00:00<00:00, 134.54it/s]

Epoch 0 | Steps 100 | Loss = 2.3699333667755127


Batch:   7%|▋         | 15/214 [00:00<00:01, 149.52it/s]

...

Batch:  13%|█▎        | 27/214 [00:00<00:01, 129.78it/s]

Epoch 49 | Steps 10500 | Loss = 1.4996896982192993


Batch:  67%|██████▋   | 143/214 [00:01<00:00, 149.40it/s]

Epoch 49 | Steps 10600 | Loss = 1.427970290184021


Batch: 100%|██████████| 214/214 [00:20<00:00, 160.55it/s]

#### Save the model checkpoint

In [ ]:
# save the model if you want